<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [188]:
from random import randint, choice
from math import exp

# A very high value to represent no direct path between cities
INT_MAX = 2147483647

# Number of cities in TSP
V = 5

# Names of the 5 cities
GENES = "ABCDE"

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10

# A list to store the population of individuals
population = []

# For simulated annealing
temperature = 10000

# Temperature threshold
temperature_threshold = 1000

# Max number of generations
max_gen = 25

In [189]:
# individual solution (a possible path)
class Individual:
  def __init__(self, gnome, fitness):
    # A string representing the path (e.g., "012340")
    self.gnome = gnome
    # The total distance of the path
    self.fitness = fitness


# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing


# Check if a character ch is already in string s
def repeat(s, ch):
  return ch in s


# Mutates a gnome by swapping two random cities in the path
# Can not swap the first city here, in the TSP, the salesman must start and end at the same city
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            temp = gnome[r]
            gnome[r] = gnome[r1]
            gnome[r1] = temp
            break
    return ''.join(gnome)


# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
      temp = rand_num(0, V)
      if str(temp) not in gnome:
        gnome += str(temp)
    return gnome


# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    mp = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
      fitness += mp[int(gnome[i])][int(gnome[i + 1])]
    return fitness


# Reduces the "temperature" as the algorithm progresses, result in less likely to accept worse solutions over time.
def cooldown(temp):
    return int(temp * 0.9)

In [190]:
# Main function for TSP problem.
def tsp_util():
    global population, temperature, temperature_threshold, max_gen  # Declare population, temperature, temperature_threshold and max_gen as global

    # Current generation number
    gen = 1

    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    # population crossing and gene mutation.
    while temperature > temperature_threshold and gen <= max_gen:
      population.sort(key=lambda x: x.fitness)
      new_population = []
      print("\nCurrent temperature: ", temperature)
      for p1 in population:
        while True:
          new_g = mutated_gene(p1.gnome)
          new_fitness = calculate_fitness(new_g)
          new_gnome = Individual(new_g, new_fitness)
          if new_gnome.fitness <= p1.fitness:
            new_population.append(new_gnome)
            break
          else:
            delta = new_fitness - p1.fitness
            prob = exp(-delta / temperature)
            if prob > 0.5:
              new_population.append(new_gnome)
              break
      temperature = cooldown(temperature)
      population = new_population

      print(f"Generation {gen}")
      print("GNOME FITNESS VALUE")

      for ind in population:
        print(f"{ind.gnome} {ind.fitness}")

      gen += 1

    # Print the smallest fitness value (shortest path) after the last iteration
    # Find the individual with the minimum fitness
    best_individual = min(population, key=lambda x: x.fitness)
    best_gnome = best_individual.gnome
    best_fitness = best_individual.fitness
    print(f"\nShortest path (smallest fitness value in the last iteration): {best_fitness}")

    # Decode the gnome (visiting order)
    city_map = {GENES[i]: i for i in range(len(GENES))}

    # Convert index string to city names
    result = "".join([GENES[int(city)] for city in best_gnome][::-1])  # List comprehension with reverse

    # Print the solution with city names (including starting city)
    print(f"Visiting order (city names): {result}{result[0]}")  # Add starting city at the end



if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
41203 4294967310
10423 13
24301 27
01342 23
10432 20
14203 4294967309
31402 4294967307
23041 2147483667
13420 2147483668
41203 4294967310

Current temperature:  10000
Generation 1
GNOME FITNESS VALUE
10324 20
10342 27
01324 16
24031 28
23140 2147483663
10423 13
31042 18
12403 24
41302 4294967314
40213 2147483664

Current temperature:  9000
Generation 2
GNOME FITNESS VALUE
12403 24
01234 19
31240 20
10342 27
10423 13
12340 22
24013 18
23041 2147483667
43210 19
41320 4294967305

Current temperature:  8100
Generation 3
GNOME FITNESS VALUE
10432 20
24031 28
01324 16
42310 16
31042 18
10342 27
12430 29
10324 20
24031 28
41302 4294967314

Current temperature:  7290
Generation 4
GNOME FITNESS VALUE
01234 19
42130 27
34012 21
10423 13
10423 13
10432 20
21034 28
24013 18
10432 20
41203 4294967310

Current temperature:  6561
Generation 5
GNOME FITNESS VALUE
10324 20
12403 24
24310 23
04231 19
10342 27
12430 29
34210 19
40132 18
21304 29
42103 21

Current 